**Changed runtime to GPU and some codes need updates**

In [1]:
import numpy as np
import pandas as pd
import os
import spacy
import string
import re
from spacy.symbols import ORTH
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

Import files

In [2]:
def unpack_dataset():
  ! mkdir -p data/aclImdb
  ! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz > .None
  ! tar -zxvf aclImdb_v1.tar.gz -C data


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
unpack_dataset()

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [5]:
!ls

aclImdb_v1.tar.gz  data  sample_data


In [6]:
# unpack dataset
from pathlib import Path
PATH = Path("data/aclImdb")
list(PATH.iterdir()) #not mkdir() but iterdir()

[PosixPath('data/aclImdb/train'),
 PosixPath('data/aclImdb/imdb.vocab'),
 PosixPath('data/aclImdb/imdbEr.txt'),
 PosixPath('data/aclImdb/README'),
 PosixPath('data/aclImdb/test')]

In [7]:
! ls ./aclImdb/train/

ls: cannot access './aclImdb/train/': No such file or directory


In [8]:
path = PATH/"train/pos/2449_7.txt"
path.read_text()

'when i saw commercials for this i was thinking "NO WHAT HAS NICK AT NITE DONE!" because it was taking up "fresh prince" slots. well, i still love the fresh prince. but george lopez is a surprisingly good show. i love how not-stereotypical benny is. carmen is a pretty good character, its really funny to see how stupid and overemotional she can be sometimes. i feel bad for the guy who plays max, he looks much younger then he actually is! but max is a fun character, and acted well. and yeah, angie is a little stereotypical, but she has her funny moments. ha ha george does have a big head! nah but he can be really good too. funny show! it definitely should be on more often then home improvement.'

Tokenization

In [9]:
# must clean up the <br space/> from text file
re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)
my_tok = spacy.load('en_core_web_sm')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [10]:
# use tokenizer
path = PATH/"train/pos/2449_7.txt"
spacy_tok(path.read_text())[:4]


['when', 'i', 'saw', 'commercials']

Vocab to index

In [11]:
pos_files = list((PATH/"train"/"pos").iterdir())
neg_files = list((PATH/"train"/"neg").iterdir())
all_files = pos_files + neg_files
all_files[:1]

[PosixPath('data/aclImdb/train/pos/5227_10.txt')]

In [12]:
'''
this code needs an update (5/11/23)
i am reading the text file in ascii that are actually not in ascii format
'''

import codecs
counts = Counter()

for path in all_files:
    # Read the text file using the appropriate encoding
    with codecs.open(path, 'r', encoding='utf-8') as file:
      text = file.read()

    # Update the counter with the tokenized text
    counts.update(spacy_tok(text))

removed any words that are less than 5 letters to remove the fillers such as 'the', 'it', 'is'....

In [13]:
len(counts.keys())

103163

In [14]:
for word in list(counts):
  if counts[word] < 5:
    del counts[word]

In [15]:
len(counts.keys())

33893

This is a code snippet in Python that creates a dictionary called vocab2index which maps words to their respective indices in a vocabulary.

The code initializes the dictionary with two special tokens: "PAD" and "UNK" with indices 0 and 1, respectively. "PAD" is typically used to pad sequences to a fixed length, and "UNK" represents unknown words that are not in the vocabulary.

The counts variable is assumed to be a dictionary that contains word counts. The code iterates over each word in counts and adds it to the vocab2index dictionary with its index. The index is the current length of the words list, which is incremented for each new word added to the list.

The words list is used to keep track of the words in the vocabulary in the order in which they were added. This is useful when converting text data to numerical format, as it allows us to maintain the original order of the words in the text.

In [16]:
vocab2index = {"": 0, "UNK": 1}
words = ["", "UNK"]
for word in counts:
  vocab2index[word] = len(words)
  words.append(word)


The encode_sentence_v1 function takes a file path, a vocabulary-to-index dictionary, and a maximum sequence length N as inputs, and returns a sequence of integer-encoded tokens representing the text in the file.

One way to improve the function for long sentences would be to take a random sample of the sentence, rather than truncating it at the Nth word. This would allow us to capture more of the sentence's content while still limiting the sequence length to N.

Here's an updated version of the function that implements this approach:


def decode_sentence_v2(path, vocab2index, N= 400, padding_start = True):
  with codecs.open(path, "r", encoding = "utf-8") as file:
    text = file.read()
    #import text to ASCII
    text = text.encode('ascii', 'ignore').decode('ascii')
    x = spacy_tok(text)

In [17]:
def encode_sentence(path, vocab2index, N= 400, padding_start = True):
  with codecs.open(path, "r", encoding = "utf-8") as file:
    text = file.read()
    x = spacy_tok(text)
    enc = np.zeros(N, dtype = np.int32)
    encl = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(encl))
    if padding_start:
      enc[: l] =encl[:l]
    else:
      enc[N-l:] = encl[: l]
    return enc, l

In [18]:
path = PATH/"train/neg/211_4.txt"
encode_sentence(path, vocab2index, N = 400, padding_start = False)

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            1,  1513,     3,  4701,     3,    31,    27,  2618,    11,
         2218,   625,    24,    20,   248,    29,   207,    33,   678,
         2685,    92,    36,  1642,    92, 13514,    14,  1696,  2232,
           14, 30419,    25,   728,    77,    34,    27,     1,   116,
      

In [19]:
class ImdbDataset(Dataset):
  def __init__(self, PATH, train = "train", N = 400, padding_start = True):
    self.path_to_images = PATH/train
    self.pos_files = list((self.path_to_images/"pos").iterdir())
    self.neg_files = list((self.path_to_images/"neg").iterdir())
    self.files = self.pos_files + self.neg_files
    self.y = np.concatenate((np.ones(len(self.pos_files), dtype = int), 
                             np.zeros(len(self.neg_files), dtype = int)), axis = 0)
    self.X = [encode_sentence(path, vocab2index, N, padding_start) for path in self.files]
    
  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    x, s = self.X[idx]
    return x, s, self.y[idx]


In [20]:
train_ds = ImdbDataset(PATH, padding_start = False)
test_ds = ImdbDataset(PATH,"test", padding_start = False)

In [21]:
batch_size = 1000
train_dl_v0 = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
valid_dl_v0 = DataLoader(test_ds, batch_size = batch_size, shuffle = False)

Debugging Our Model

In [22]:
batch_size = 7
x, s, y = next(iter(train_dl_v0))
x.shape, s.shape

(torch.Size([1000, 400]), torch.Size([1000]))

In [23]:
train_ds[2]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

Understanding LSTMs

```
# Input dim is the dimension of the embedding for each word (2 in the example)
# Output dim is the dimension of the hidden layer (4 in this example)
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature).
```



In [24]:
lstm = nn.LSTM(2, 4, batch_first = True)

The resulting tensor is then reshaped using the view() function. The view() function creates a new tensor with the same data as the original tensor but with a different shape. In this case, the resulting tensor is reshaped to have dimensions of (1, len(inputs), -1), where 1 is the batch size, len(inputs) is the length of the original list, and -1 is inferred from the size of the input tensors to ensure that the total number of elements in the reshaped tensor is the same as the original tensor.

In [25]:
inputs = [torch.randn(1, 2) for _ in range(5)]
inputs = torch.cat(inputs).view(1, len(inputs), -1)
inputs

tensor([[[ 0.0185, -0.6603],
         [ 0.0416,  0.5944],
         [-0.4851,  0.7470],
         [ 1.5930,  0.1524],
         [-0.5743, -0.0199]]])

```
# RNNs assume this input shape
# input shape should be  bash_size x seq_len x embedding dimension (if batch_first=True)
```

In [26]:
inputs.shape

torch.Size([1, 5, 2])

In [27]:
out, (hidden, cell) = lstm(inputs)

In [28]:
print(out.shape)
out

torch.Size([1, 5, 4])


tensor([[[-0.0656, -0.0208, -0.0340,  0.1313],
         [-0.0443, -0.0344,  0.0408,  0.2059],
         [-0.0447, -0.0195,  0.1110,  0.1853],
         [ 0.0360, -0.0188,  0.0282,  0.3232],
         [-0.0447,  0.0015,  0.0132,  0.2404]]], grad_fn=<TransposeBackward0>)

Model 0

In [29]:
class LSTMModel(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim):
    super(LSTMModel, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embed_dim, padding_idx = 0)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first = True)
    self.linearOut = nn.Linear(hidden_dim, 1)
    self.dropout = nn.Dropout(0.5)

#this is forward here
  def forward(self,x):
    x = self.embeddings(x)
    x = self.dropout(x)
    out_pack, (ht, ct) = self.lstm(x)
    return self.linearOut(ht[-1])

Debugging

In [30]:
batch_size = 7
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
x, s, y = next(iter(train_dl))
x.shape, s.shape

(torch.Size([7, 400]), torch.Size([7]))

sort the batch so we can use pack_padded_sequence

In [31]:
s, index = s.sort(0, descending = True)
x = x[index]
s

tensor([400, 400, 226, 192, 143, 122,  48])

when I ran x.shape below, i got an error. The error suggests that there is an issue with decoding a byte string containing non-ASCII characters when trying to print the shape of the variable x.

To resolve this issue, you can explicitly convert the x.shape output to a string using the str() function, which will handle the encoding properly. Here's an example of how you can modify your code:

In [32]:
vocab_size = len(words)
embed_dim = 10
embed = nn.Embedding(vocab_size, embed_dim, padding_idx = 0)

In [33]:
x = embed(x.long())
x.shape

torch.Size([7, 400, 10])

The [:, -1] slicing operation selects all rows (:) and the last column (-1) of the tensor. It effectively extracts the elements in the last column of out while preserving the dimensions of the tensor.

In [34]:
from torch.nn.modules import padding
hidden_dim = 9
lstm = nn.LSTM(embed_dim, hidden_dim, batch_first = True)

In [35]:
x_pack = pack_padded_sequence(x, list(s), batch_first = True)

In [36]:
out, (ht, ct) = lstm(x_pack)

In [37]:
ht.shape

torch.Size([1, 7, 9])

In [38]:
# a vector of size 9 (hidden_dim) for every element on the batch
ht[-1].shape

torch.Size([7, 9])

In [39]:
linear = nn.Linear(hidden_dim, 1)
y_hat = linear(ht[-1])
y_hat

tensor([[0.3694],
        [0.3781],
        [0.4155],
        [0.3261],
        [0.3261],
        [0.3261],
        [0.3261]], grad_fn=<AddmmBackward0>)

In [40]:
index.unsqueeze(1).shape

torch.Size([7, 1])

In [41]:
#takes back to original ordering
h = torch.zeros_like(y_hat).scatter_(0, index.unsqueeze(1), y_hat)

In [42]:
h

tensor([[0.3261],
        [0.3261],
        [0.3261],
        [0.4155],
        [0.3694],
        [0.3781],
        [0.3261]], grad_fn=<ScatterBackward0>)

In [43]:
index

tensor([4, 5, 3, 1, 2, 6, 0])

The underscore _ is used as a placeholder to ignore or discard the second element returned by the model. It is common to use an underscore when you're not interested in a particular value returned by a function or method.

The line y_pred, _ = model(x) allows you to obtain the predicted output y_pred from the model while ignoring or discarding any other values that might be returned. This assumes that the model's forward method is designed to return multiple values, and you are only interested in the first value, which represents the predicted output.

In [44]:
def train_epochs(model, epochs = 10, lr = 0.001):
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = torch.optim.Adam(parameters, lr = lr)
  for i in range(epochs):
    model.train()
    sum_loss = 0.0
    total = 0
    # 5/23/23 debugged here
    for x, s, y in train_dl:
      x = x.long().cuda()
      y = y.float().cuda()
      y_pred = model(x)
      optimizer.zero_grad()
      loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
      loss.backward()
      optimizer.step()
      sum_loss += loss.item() * y.shape[0]
      total += y.shape[0]
    val_loss, val_acc = val_metrics_v0(model, val_dl)
    if i %5 == 1:
      print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss / total, val_loss, val_acc))


loss.item() retrieves the scalar value of the loss tensor. The item() method is used to extract the loss value as a Python number.

y.shape[0] gives the batch size, which represents the number of samples in the current batch.

The product of loss.item() and y.shape[0] gives the loss value multiplied by the batch size, which represents the cumulative loss for the current batch.

sum_loss += loss.item() * y.shape[0] accumulates the cumulative loss for each batch by adding it to the sum_loss variable.

In [45]:
def val_metrics_v0(model, valid_dl):
  model.eval()
  correct = 0
  total = 0
  sum_loss = 0.0
  for x, s,  y in valid_dl:
    x = x.long().cuda()
    y = y.float().cuda().unsqueeze(1)
    y_hat = model(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct += (y_pred.float() == y).float().sum()
    total += y.shape[0]
    sum_loss += loss.item() * y.shape[0]
  return sum_loss / total, correct / total



In [46]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(test_ds, batch_size = batch_size)

In [47]:
x, s, y = next(iter(train_dl))
x.shape, s.shape

(torch.Size([1000, 400]), torch.Size([1000]))

In [48]:
vocab_size = len(words)
print(vocab_size)
model_v0 = LSTMModel(vocab_size, 50, 50).cuda()

33895


In [49]:
train_epochs(model_v0, epochs = 20, lr = 0.01)

train loss 0.662 val loss 0.619 and val accuracy 0.662
train loss 0.413 val loss 0.563 and val accuracy 0.778
train loss 0.218 val loss 0.510 and val accuracy 0.843
train loss 0.147 val loss 0.502 and val accuracy 0.860


In [50]:
def save_model(m, p): torch.save(m.state_dict(), p)
def load_model(m, p): m.load_state_dict(torch.load(p))

In [51]:
p = PATH/"model-1.pth"
save_model(model_v0, p)

Model 1

In [52]:
class LSTMModel_1(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim):
    super(LSTMModel_1, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embed_dim, padding_idx = 0)
    self.hidden_dim = hidden_dim
    self.dropout = nn.Dropout(0.5)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first = True)
    self.linear = nn.Linear(hidden_dim, 1)
  
  def forward(self, x, s):
    s, sort_index = torch.sort(s, 0, descending = True)
    s = s.numpy().tolist()
    x = x[sort_index]
    x = self.embeddings(x)
    x = self.dropout(x)
    x_pack = pack_padded_sequence(x, s, batch_first = True)
    out_pack, (ht, ct) = self.lstm(x_pack)
    out = self.linear(ht[-1])
    return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out)



In [85]:
def train_epochs_v1(model, epochs = 10, lr = 0.01):
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = torch.optim.Adam(parameters, lr = lr)
  for i in range(epochs):
    model.train()
    sum_loss = 0.0
    total = 0
    for x, s, y in train_dl:
      x = x.long().cuda()
      y = y.float().cuda()
      y_pred = model(x, s)
      optimizer.zero_grad()
      loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
      loss.backward()
      optimizer.step() #used to update the gradient
      sum_loss += loss.item() * y.shape[0]
      total += y.shape[0]
    val_loss, val_acc = val_metrics(model, valid_dl)
    if i % 5 == 1:
      print('total loss %.3f val loss %.3f and val accuracy %.3f' % (sum_loss /total, val_loss, val_acc))




In [86]:
def val_metrics(model, valid_dl):
  model.eval()
  correct = 0
  total = 0
  sum_loss = 0.0
  for x, s, y in valid_dl:
    x = x.long().cuda()
    y = y.float().unsqueeze(1).cuda()
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct += (y_pred.float() == y).float().sum()
    total += y.shape[0]
    sum_loss += loss.item() * y.shape[0]
  return sum_loss /total, correct / total


In [96]:
# dataset with padding at the end
train_ds_v1 = ImdbDataset(PATH)
valid_ds_v1 = ImdbDataset(PATH, "test")

In [97]:
batch_size = 2000
train_dl = DataLoader(train_ds_v1, batch_size= batch_size, shuffle = True)
valid_dl = DataLoader(valid_ds_v1, batch_size= batch_size)

In [98]:
vocab_size = len(words)
print(vocab_size)
model = LSTMModel_1(vocab_size, 50, 50).cuda()

33895


In [99]:
train_epochs_v1(model, epochs = 30, lr = 0.01)

total loss 0.675 val loss 0.662 and val accuracy 0.599
total loss 0.488 val loss 0.695 and val accuracy 0.729
total loss 0.495 val loss 0.663 and val accuracy 0.717
total loss 0.288 val loss 0.648 and val accuracy 0.779
total loss 0.201 val loss 0.539 and val accuracy 0.832
total loss 0.144 val loss 0.498 and val accuracy 0.858


## GRUModel with Dropout

In [100]:
class GruModel(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim):
    super(GruModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.embeddings = nn.Embedding(vocab_size, hidden_dim, padding_idx = 0)
    self.dropout = nn.Dropout(0.5)
    self.gru = nn.GRU(embed_dim, hidden_dim, batch_first = True)
    self.linear = nn.Linear(hidden_dim, 1)
  
  def forward(self, x, s):
    s, sort_index = torch.sort(s, 0, descending = True)
    s = s.numpy().tolist()
    x = x[sort_index]
    x = self.embeddings(x)
    x = self.dropout(x)
    x_pack = pack_padded_sequence(x, list(s), batch_first = True)
    out_pack, ht = self.gru(x_pack)
    out = self.linear(ht[-1])
    return torch.zeros_like(out).scatter_(0, sort_index.unsqueeze(1).cuda(), out)
    

In [101]:
vocab_size = len(words)
print(vocab_size)
model2 = GruModel(vocab_size, 50, 50).cuda()

33895


In [103]:
train_epochs_v1(model2, epochs = 30, lr = 0.001)

total loss 0.337 val loss 0.927 and val accuracy 0.759
total loss 0.307 val loss 1.021 and val accuracy 0.758
total loss 0.284 val loss 1.018 and val accuracy 0.769
total loss 0.267 val loss 0.994 and val accuracy 0.781
total loss 0.255 val loss 0.991 and val accuracy 0.789
total loss 0.238 val loss 0.967 and val accuracy 0.799


In [104]:
p = PATH/"model-2.pth"
save_model(model2, p)

##Bidirectional LSTM

In [74]:
class LSTMBiModel(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim):
    super(LSTMBiModel, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embed_dim, padding_idx= 0)
    self.hidden_dim = hidden_dim
    self.lstm = nn.LSTM( embed_dim, hidden_dim, batch_first = True, dropout = 0.3,
                        num_layers = 2, bidirectional = True)
    self.linear = nn.Linear(2 * hidden_dim, 1)
  
  def forward(self, x, s):
    # s is the length of the sentence
    s, sort_index = torch.sort(s, 0, descending = True)
    s = s.numpy().tolist()
    x = x[sort_index]
    x = self.embeddings(x)
    x_pack = pack_padded_sequence(x, s, batch_first= True)
    out_pack, (ht, ct) = self.lstm(x_pack)
    h = torch.cat((ht[-2,:,:], ht[-1, :, :]), dim = 1)
    h = self.linear(h)

    return torch.zeros_like(h).scatter_(0, sort_index.unsqueeze(1).cuda(), h)


In [105]:
vocab_size = len(words)
model3 = LSTMBiModel(vocab_size, 50, 50).cuda()

In [107]:
train_epochs_v1(model3, epochs = 15, lr = 0.01)

total loss 0.563 val loss 0.590 and val accuracy 0.693
total loss 0.141 val loss 0.495 and val accuracy 0.814
total loss 0.022 val loss 1.013 and val accuracy 0.808


In [108]:
p = PATH/"model-3.pth"
save_model(model3, p)